# 심전도 질의 qa

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader('./data/construcnt_law.pdf')
documents = loader.load()
documents

[Document(metadata={'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2025-03-12T17:33:04+09:00', 'moddate': '2025-03-12T17:33:04+09:00', 'source': './data/construcnt_law.pdf', 'total_pages': 69, 'page': 0, 'page_label': '1'}, page_content='법제처                                                            1                                                       국가법령정보센터\n건축법 시행령\n \n건축법 시행령\n[시행 2025. 1. 21.] [대통령령 제35221호, 2025. 1. 21., 타법개정]\n국토교통부 (건축정책과 - 건축제도 일반) 044-201-3762, 3763\n국토교통부 (건축안전과 - 건축구조 규정 운영) 044-201-4991\n국토교통부 (녹색건축과 - 건축설비ㆍ조경 규정 운영) 044-201-4753\n국토교통부 (건축안전과 - 피난ㆍ마감재료 규정 운영) 044-201-4992\n국토교통부 (건축정책과 - 건축감리 규정 운영) 044-201-4752\n국토교통부 (건축정책과 - 위반건축물 규정 운영) 044-201-3762, 3761\n       제1장 총칙\n \n제1조(목적) 이 영은 「건축법」에서 위임된 사항과 그 시행에 필요한 사항을 규정함을 목적으로 한다.\n[전문개정 2008. 10. 29.]\n \n제2조(정의) 이 영에서 사용하는 용어의 뜻은 다음과 같다. <개정 2009. 7. 16., 2010. 2. 18., 2011. 12. 8., 2011. 12. 30.,\n2013. 3. 23., 2014. 11. 11., 2014. 11. 28., 2015. 9. 22., 2016. 1. 19., 2

In [6]:
import re
def normalize_whitespace(text):
    return re.sub(r'\s{5,}', ' ', text)

for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')
    doc.page_content = normalize_whitespace(doc.page_content)

len(documents)

    

69

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=350, 
    chunk_overlap=20 
    )
docs = splitter.split_documents(documents)
len(docs)

410

In [10]:
# 임베딩 모델 생성
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [16]:
from langchain_chroma.vectorstores import Chroma
vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='./cunstruct_law_vector_store'
)
vector_store.add_documents(docs)

['1cd2798f-02ad-4caf-971d-207f1ad81778',
 '1654bb4c-8232-4845-9696-456caaae2a05',
 '49124f75-ea7b-4b78-ac7b-52bbc604e185',
 'e7869885-25e5-433a-af3d-2765aeb9f8d2',
 'f27610d3-0ba7-4060-aa9d-675c90a74620',
 'e4a031cd-86e8-42fa-92e4-354761d3bddf',
 '4d1425a5-cee7-458a-9781-4f3f716bdabd',
 'd0422a49-ea78-4369-aa68-beaf17880531',
 '0e3762c1-7ec1-4d75-bcfc-420d661e887d',
 'c3e3a6a1-10c3-4419-9033-0146e4d128c1',
 '56b08442-a59e-4db7-ac84-779bc6771112',
 'cc3bd7df-b30f-462c-b3cb-ee2f1ecc8c77',
 'e06c9b48-dd98-4ac6-b41d-bde0643aafaf',
 'b55056a4-0153-425c-8707-8297edd413cb',
 '0dd889d6-fe82-4228-9c8f-f31851e00d6a',
 '19bc011c-2284-457a-af40-035f2bb4763a',
 '75f89117-4e7b-4c9b-a74f-68807b2ecdb7',
 'b79375e4-7d46-4ec5-97da-c7866ca48a47',
 'f7ce774d-19b0-424b-a365-61003e5cba89',
 'b602a9a5-86da-442c-a213-f068af7e36cf',
 'f5dc9e6a-7da4-4bc1-8725-676dc61b9e01',
 'ede54b1d-154d-4c32-b403-d3e992948d5e',
 '0118f3b3-4ec1-46fa-b6be-2be4f4b6ac91',
 '9c341a68-784f-4bdc-8f09-07f43b4396c6',
 'f914e88b-3667-

In [17]:
from langchain_openai import ChatOpenAI
# llm 설정 (모델 생성)
model = ChatOpenAI(
    model = 'gpt-4o-mini',
    temperature=0.5
)

In [22]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    template="""
    너는 건축법률 상담사야 
    
    다음 '질문'에 대해 '문서'의 내용만을 참고하여 '답변'을 생성해 주세요.
    질문 : {question}
    문서 : {context}
    답변 : 
    해결법 : 
    
    <답변에 대한 조건>
    - 답변에는 어떤 법률을 어겼는지 정확하게 명시할 것
    - 해결법도 도출해줄것
    """,
    input_variables={'question','context'}
)

In [24]:
retriever = vector_store.as_retriever()

In [25]:
from langchain.chains import RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    chain_type='stuff',
    retriever=retriever,
    chain_type_kwargs={'prompt':prompt}
)

In [26]:
qa_chain('A씨는 자신의 땅에 창고를 짓고 싶어 건축물을 신축했다. 하지만 건축허가를 받지 않고 공사를 진행했다. 이 경우 A씨가 위반한 건축법상 규정은 무엇일까?')

{'query': 'A씨는 자신의 땅에 창고를 짓고 싶어 건축물을 신축했다. 하지만 건축허가를 받지 않고 공사를 진행했다. 이 경우 A씨가 위반한 건축법상 규정은 무엇일까?',
 'result': 'A씨가 위반한 건축법상 규정은 건축허가를 받지 않고 건축물을 신축한 것으로, 이는 건축법 제11조에 따른 건축허가 규정을 위반한 것입니다. 건축법 제11조는 건축물을 신축하기 위해서는 반드시 해당 관청으로부터 건축허가를 받아야 한다고 명시하고 있습니다.\n\n해결법:\nA씨는 이미 진행한 공사를 중단하고, 해당 관청에 건축허가를 신청해야 합니다. 허가가 승인되면, 법적 절차에 따라 공사를 재개할 수 있습니다. 만약 허가를 받지 않은 상태로 계속 공사를 진행할 경우, 행정처분이나 벌금 등의 법적 제재를 받을 수 있으므로, 신속하게 조치를 취하는 것이 중요합니다.'}